In [ ]:
# 這邊結合網路爬蟲並將資料寫入 MySQL 資料庫儲存以供分析

# 1. 爬蟲部分

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

driver = webdriver.Chrome()

driver.get("https://www.tpex.org.tw/zh-tw/mainboard/trading/realtime/advance.html")

def fetch_data(i) : 

    wait = WebDriverWait(driver,5)
    wait.until(expected_conditions.visibility_of_element_located((By.CLASS_NAME,"table")))

    table = driver.find_element(By.CLASS_NAME,"table")

    data = []

    rows = table.find_elements(By.TAG_NAME,"tr")
    
    if i == 0 : 

        for row in rows : 

            th_cells = row.find_elements(By.TAG_NAME,"th")

            if th_cells : 

                th_row = [th_cell.text for th_cell in th_cells]

                data.append(th_row)

        for row in rows : 

            td_cells = row.find_elements(By.TAG_NAME,"td")

            if td_cells : 

                td_row = [td_cell.text for td_cell in td_cells]

                data.append(td_row)

    else : 

        for row in rows : 

            td_cells = row.find_elements(By.TAG_NAME,"td")

            if td_cells : 

                td_row = [td_cell.text for td_cell in td_cells]

                data.append(td_row)

    return data

def click_next() : 

    wait = WebDriverWait(driver,5)
    wait.until(expected_conditions.visibility_of_element_located((By.LINK_TEXT,"下一頁")))

    page = driver.find_element(By.LINK_TEXT,"下一頁")

    page.click()

def count_next() : 

    wait = WebDriverWait(driver,5)
    wait.until(expected_conditions.visibility_of_element_located((By.LINK_TEXT,"下一頁")))

    table = driver.find_element(By.CLASS_NAME,"table")

    li = table.find_elements(By.TAG_NAME,"li")

    count = len(li) - 4

    return count

all_data = []

count_clicks = count_next()

for i in range(count_clicks) : 

    all_data.extend(fetch_data(i))

    if i == count_clicks - 1 : 

        break

    click_next()

column_row , *data_row = all_data

# 2. 寫入 MySQL 部分

import pymysql

connection = pymysql.connect(

    host="127.0.0.1",
    user="root",
    database="example",
    password="12345",
    charset="utf8"

)

cursor = connection.cursor()

# 如果有括號這種特殊字元在 SQL 指令裡面，就要把指令用 """ """ 包起來，然後有括號的欄位就要用 ` ` 包起來，這樣才能查詢

# 目前還找不到有百分比符號的解決方法，盡量就是資料庫欄位和這邊指令都不要出現

sql = """INSERT INTO new_table_4 (`代號`,`名稱`,`收盤價(元)`,`上漲金額(元)`,`漲幅百分比`) VALUES (%s,%s,%s,%s,%s)"""

try : 

    cursor.executemany(sql,data_row)

    connection.commit()

except Exception as e : 

    print("寫入失敗",e)

    connection.rollback()

cursor.close()

connection.close()

# 3. 讀取出來變成 List of List 部分

connection = pymysql.connect(

    host="127.0.0.1",
    user="root",
    database="example",
    password="12345",
    charset="utf8"

)

cursor = connection.cursor()

sql = "SELECT * FROM new_table_4"

cursor.execute(sql)

results = cursor.fetchall()

for result in results : 

    print(result)


('2718', '全心投控', '55.20', '+5.00', '+9.96%')
('3066', '李洲', '19.25', '+1.45', '+8.14%')
('3071', '協禧', '22.70', '+2.05', '+9.92%')
('3152', '璟德', '131.00', '+11.50', '+9.62%')
('3213', '茂訊', '114.00', '+8.50', '+8.05%')
('3227', '原相', '211.50', '+19.00', '+9.87%')
('3323', '加百裕', '42.10', '+3.35', '+8.64%')
('3430', '奇鈦科', '50.70', '+4.45', '+9.62%')
('3483', '力致', '130.00', '+11.50', '+9.70%')
('3555', '博士旺', '54.80', '+4.95', '+9.92%')
('4534', '慶騰', '18.75', '+1.70', '+9.97%')
('4931', '新盛力', '152.00', '+13.50', '+9.74%')
('5302', '太欣', '10.80', '+0.88', '+8.87%')
('5314', '世紀*', '87.30', '+6.90', '+8.58%')
('5475', '德宏', '44.30', '+4.00', '+9.92%')
('6294', '智基', '59.50', '+4.40', '+7.98%')
('6418', '詠昇', '26.65', '+2.40', '+9.89%')
('6441', '廣錠', '29.00', '+2.50', '+9.43%')
('6498', '久禾光', '128.50', '+10.50', '+8.89%')
('6499', '益安', '112.00', '+10.00', '+9.80%')
('6697', '東捷資訊', '55.40', '+3.90', '+7.57%')
('6703', '軒郁', '121.50', '+11.00', '+9.95%')
('6727', '亞泰金屬', '150.50', '+